<a href="https://colab.research.google.com/github/MBUYt0n/blind-eye-dealers/blob/main/wrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'emogen2/tensorflow2/one/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F53721%2F64424%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240613%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240613T132419Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da274f4e22fc82aa42ef2bd6731c6d2033b1d6ecf9240ba2a39989384ef7f6a49300e2c38c10c4cb235078f88bd815acf60b8249e16ce719a95388786cfc98f5d97802a3cd7824d16ecdf6acec387a6d0b223959fda03c3a0e95e1f0dfea8fefc83e4db5de89a7dd1fb3b241c94b2c05f53d6546f4292342b19f9afdb98a4f524365e3a2a7a9b7dbec8c1030d23b5a436a406a9fef7029251fd1521670b2f364fd2634c412170fd85cb622978eed2ff3348ed258ddabfd95fd72984968aa01802b4557768f3c6868c7b72f5c53fa5a152e8a9fea3f55285f1d0715b254b38fb0a338b0a135635f4690d81724ce333ac67c203165224b401e17f9296c433be87bc,yolov8/pytorch/open-image-v8/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F50333%2F60178%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240613%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240613T132420Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1e6139bd454ddc39527c6e4062138f8b3b2ce1f8a454ae59f4952c82199e4cd67c811b5d4214568f25602af5657b683f1e7624683aa23c6324e4b48f3af49be303a8a6e11a4e64318078671019067e9154e3e1be9b615d3d3a0030953e66facbb9392332a34cc69bea67d7d6234da21d6ef240e8a779e787c3c1c6858593e238a2f24e667ded02600c6feffb78195034cc8bc2457a925790469787bf3d2299179624020d0344aad29315c5f9bab2cb9c2f860fb22c836c6ea9093f32f979eb7dd5420485c25b919581e693f496031c7b17c9acf3a08de1d02dcc353ef91c34510a6f872894767485aa0b0da00565f63aa90e2dd32eb5f3ede6a401c8ffb1b2a7'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 203010682 bytes downloaded
Downloaded and uncompressed: emogen2/tensorflow2/one/1
[==================================================] 21264265 bytes downloaded
Downloaded and uncompressed: yolov8/pytorch/open-image-v8/1
Data source import complete.


In [6]:
!pip install ultralytics lapx paddleocr paddlepaddle pytube dlib

In [7]:
import logging
import sys
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Activation, add, Dense, Flatten, Dropout, Conv2D, AveragePooling2D, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
sys.setrecursionlimit(2 ** 20)
np.random.seed(2 ** 10)


class WideResNet:
    def __init__(self, image_size, depth=16, k=8):
        self._depth = depth
        self._k = k
        self._dropout_probability = 0
        self._weight_decay = 0.0005
        self._use_bias = False
        self._weight_init = "he_normal"

        if K.image_data_format() == "channels_first":
            logging.debug("image_dim_ordering = 'th'")
            self._channel_axis = 1
            self._input_shape = (3, image_size, image_size)
        else:
            logging.debug("image_dim_ordering = 'tf'")
            self._channel_axis = -1
            self._input_shape = (image_size, image_size, 3)

    # Wide residual network http://arxiv.org/abs/1605.07146
    def _wide_basic(self, n_input_plane, n_output_plane, stride):
        def f(net):
            # format of conv_params:
            #               [ [kernel_size=("kernel width", "kernel height"),
            #               strides="(stride_vertical,stride_horizontal)",
            #               padding="same" or "valid"] ]
            # B(3,3): orignal <<basic>> block
            conv_params = [[3, 3, stride, "same"],
                           [3, 3, (1, 1), "same"]]

            n_bottleneck_plane = n_output_plane

            # Residual block
            for i, v in enumerate(conv_params):
                if i == 0:
                    if n_input_plane != n_output_plane:
                        net = BatchNormalization(axis=self._channel_axis)(net)
                        net = Activation("relu")(net)
                        convs = net
                    else:
                        convs = BatchNormalization(axis=self._channel_axis)(net)
                        convs = Activation("relu")(convs)

                    convs = Conv2D(n_bottleneck_plane, kernel_size=(v[0], v[1]),
                                          strides=v[2],
                                          padding=v[3],
                                          kernel_initializer=self._weight_init,
                                          kernel_regularizer=l2(self._weight_decay),
                                          use_bias=self._use_bias)(convs)
                else:
                    convs = BatchNormalization(axis=self._channel_axis)(convs)
                    convs = Activation("relu")(convs)
                    if self._dropout_probability > 0:
                        convs = Dropout(self._dropout_probability)(convs)
                    convs = Conv2D(n_bottleneck_plane, kernel_size=(v[0], v[1]),
                                          strides=v[2],
                                          padding=v[3],
                                          kernel_initializer=self._weight_init,
                                          kernel_regularizer=l2(self._weight_decay),
                                          use_bias=self._use_bias)(convs)

            # Shortcut Connection: identity function or 1x1 convolutional
            #  (depends on difference between input & output shape - this
            #   corresponds to whether we are using the first block in each
            #   group; see _layer() ).
            if n_input_plane != n_output_plane:
                shortcut = Conv2D(n_output_plane, kernel_size=(1, 1),
                                         strides=stride,
                                         padding="same",
                                         kernel_initializer=self._weight_init,
                                         kernel_regularizer=l2(self._weight_decay),
                                         use_bias=self._use_bias)(net)
            else:
                shortcut = net

            return add([convs, shortcut])

        return f


    # "Stacking Residual Units on the same stage"
    def _layer(self, block, n_input_plane, n_output_plane, count, stride):
        def f(net):
            net = block(n_input_plane, n_output_plane, stride)(net)
            for i in range(2, int(count + 1)):
                net = block(n_output_plane, n_output_plane, stride=(1, 1))(net)
            return net

        return f

#    def create_model(self):
    def __call__(self):
        logging.debug("Creating model...")

        assert ((self._depth - 4) % 6 == 0)
        n = (self._depth - 4) / 6

        inputs = Input(shape=self._input_shape)

        n_stages = [16, 16 * self._k, 32 * self._k, 64 * self._k]

        conv1 = Conv2D(filters=n_stages[0], kernel_size=(3, 3),
                              strides=(1, 1),
                              padding="same",
                              kernel_initializer=self._weight_init,
                              kernel_regularizer=l2(self._weight_decay),
                              use_bias=self._use_bias)(inputs)  # "One conv at the beginning (spatial size: 32x32)"

        # Add wide residual blocks
        block_fn = self._wide_basic
        conv2 = self._layer(block_fn, n_input_plane=n_stages[0], n_output_plane=n_stages[1], count=n, stride=(1, 1))(conv1)
        conv3 = self._layer(block_fn, n_input_plane=n_stages[1], n_output_plane=n_stages[2], count=n, stride=(2, 2))(conv2)
        conv4 = self._layer(block_fn, n_input_plane=n_stages[2], n_output_plane=n_stages[3], count=n, stride=(2, 2))(conv3)
        batch_norm = BatchNormalization(axis=self._channel_axis)(conv4)
        relu = Activation("relu")(batch_norm)

        # Classifier block
        pool = AveragePooling2D(pool_size=(8, 8), strides=(1, 1), padding="same")(relu)
        flatten = Flatten()(pool)
        predictions_g = Dense(units=2, kernel_initializer=self._weight_init, use_bias=self._use_bias,
                              kernel_regularizer=l2(self._weight_decay), activation="softmax",
                              name="pred_gender")(flatten)
        predictions_a = Dense(units=101, kernel_initializer=self._weight_init, use_bias=self._use_bias,
                              kernel_regularizer=l2(self._weight_decay), activation="softmax",
                              name="pred_age")(flatten)
        model = Model(inputs=inputs, outputs=[predictions_g, predictions_a])

        return model

In [8]:
from pytube import YouTube

video_url = 'https://www.youtube.com/watch?v=o4Zd-IeMlSY'

yt = YouTube(video_url)

video_stream = yt.streams.get_highest_resolution()

output_path = 'downloaded_video.mp4'

video_stream.download(filename=output_path)

print(f"Video downloaded and saved as {output_path}")

Video downloaded and saved as downloaded_video.mp4


In [17]:
from tensorflow.keras.models import load_model
from ultralytics import YOLO
from paddleocr import PaddleOCR
import time
import dlib
import cv2
from tensorflow.keras.preprocessing.image import img_to_array
import sys
import io
from transformers import pipeline
import hashlib

class WrapperClass:
    def __init__(self, img_size, depth, k, margin):
        self.img_size = img_size
        self.margin = margin
        self.ag_model = WideResNet(img_size, depth=depth, k=k)()
        self.ag_model.load_weights("/kaggle/input/emogen2/tensorflow2/one/1/weights.28-3.73.hdf5")
        self.emotion_model = load_model("/kaggle/input/emogen2/tensorflow2/one/1/emotion_little_vgg_2.h5")
        self.yolo = YOLO("/kaggle/input/yolov8/pytorch/open-image-v8/1/yolov8s-oiv7.pt")
        self.ocr = PaddleOCR(use_angle_cls=True, lang='en') # need to run only once to download and load model into memory
        self.emotion_classes = {0:"Angry", 1:"Disgust", 2:"Fear", 3:"Happy", 4:"Sad", 5:"Surprise", 6:"Neutral"}
        self.detector = dlib.get_frontal_face_detector()
        self.corrector = pipeline('text2text-generation', model='vennify/t5-base-grammar-correction')
        self.prev_frame = None

    def hash_frame(self, frame):
        frame_bytes = frame.tobytes()
        return hashlib.md5(frame_bytes).hexdigest()

    def detection(self, frame):
        return self.detector(frame, 1)

    def preprocess_face(self, img_h, img_w, detected, faces, frame):
        preprocessed_faces_emo = []
        if len(detected) > 0:
            for i, d in enumerate(detected):
                x1, y1, x2, y2, w, h = d.left(), d.top(), d.right() + 1, d.bottom() + 1, d.width(), d.height()
                xw1 = max(int(x1 - self.margin * w), 0)
                yw1 = max(int(y1 - self.margin * h), 0)
                xw2 = min(int(x2 + self.margin * w), img_w - 1)
                yw2 = min(int(y2 + self.margin * h), img_h - 1)
                faces[i, :, :, :] = cv2.resize(frame[yw1:yw2 + 1, xw1:xw2 + 1, :], (self.img_size, self.img_size))
                face =  frame[yw1:yw2 + 1, xw1:xw2 + 1, :]
                face_gray_emo = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
                face_gray_emo = cv2.resize(face_gray_emo, (48, 48), interpolation = cv2.INTER_AREA)
                face_gray_emo = face_gray_emo.astype("float") / 255.0
                face_gray_emo = img_to_array(face_gray_emo)
                face_gray_emo = np.expand_dims(face_gray_emo, axis=0)
                preprocessed_faces_emo.append(face_gray_emo)

        return preprocessed_faces_emo, faces

    def predict_ages(self, detected, faces):
        if len(detected) > 0:
            results = self.ag_model.predict(np.array(faces))
            predicted_genders = results[0]
            predicted_ages = np.argsort(results[1])[0][-5:]
            age_range = (min(predicted_ages), max(predicted_ages))

            return predicted_genders, age_range

        return None, None

    def predict_emo(self, detected, preprocessed_faces_emo):
        emo_labels = []
        for i, d in enumerate(detected):
            preds = self.emotion_model.predict(preprocessed_faces_emo[i])[0]
            e = self.emotion_classes[preds.argmax()]
            emo_labels.append(e)
        return emo_labels

    def yolo_pred(self, frame):
        res = self.yolo(frame)
        return [(i['name'], i['box']) for j in res for i in j.summary()]

    def correct_text(self, text):
        return self.corrector(text)[0]['generated_text']


    def vision_pred(self, frame):
        preprocessed_faces_emo = []
        input_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img_h, img_w, _ = np.shape(input_img)
        detected = self.detector(frame, 1)
        faces = np.empty((len(detected), self.img_size, self.img_size, 3))
        preprocessed_faces_emo, faces = self.preprocess_face(img_h, img_w, detected, faces, frame)
        gender, age = self.predict_ages(detected, faces)
        emo_labels = self.predict_emo(detected, preprocessed_faces_emo)
        print(gender, age, emo_labels)
        result = self.yolo_pred(frame)
        print(result)

    def ocr_pred(self, frame):
        frame_hash = self.hash_frame(frame)
        if frame_hash == self.prev_frame:
            print("skipping frame")
            return
        result = self.ocr.ocr(frame, cls=True)
        detected_text = ""
        for idx in range(len(result)):
            res = result[idx]
            if res:
                for line in res:
                    detected_text += " " + line[1][0]
        # corrected_text = self.correct_text(detected_text)
        print(detected_text)
        self.prev_frame = frame_hash

    def make_prediction(self, video_path, func):
        a = time.time()
        output_video_path = 'output.mp4'
        cap = cv2.VideoCapture(video_path)
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        frame_count = 0

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            if frame_count % 30 != 0:
                frame_count += 1
                continue
            if frame_count == 600:
              break

            print(frame_count)
            func(frame)
            frame_count += 1

        cap.release()
        cv2.destroyAllWindows()
        b = time.time()
        print(b - a)

    def __call__(self, video_path, func):
        if func == "see":
            self.make_prediction(video_path, self.vision_pred)
        elif func == "ocr":
            self.make_prediction(video_path, self.ocr_pred)

In [18]:
import logging
import warnings
logging.basicConfig(level=logging.ERROR)

# Suppress all warnings
warnings.filterwarnings("ignore")

In [21]:
a = WrapperClass(64, 16, 8, 0.3)
a("/content/downloaded_video.mp4", "see")

[2024/06/14 01:15:10] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_char_dict_path='

In [20]:
a("/content/downloaded_video.mp4", "ocr")

0
[2024/06/14 01:14:07] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.1559889316558838
[2024/06/14 01:14:07] ppocr DEBUG: cls num  : 1, elapsed : 0.016289472579956055
[2024/06/14 01:14:08] ppocr DEBUG: rec_res num  : 1, elapsed : 0.0923764705657959

30
[2024/06/14 01:14:08] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.16633033752441406
[2024/06/14 01:14:08] ppocr DEBUG: cls num  : 1, elapsed : 0.009167909622192383
[2024/06/14 01:14:08] ppocr DEBUG: rec_res num  : 1, elapsed : 0.050390005111694336

60
[2024/06/14 01:14:09] ppocr DEBUG: dt_boxes num : 2, elapsed : 0.2316126823425293
[2024/06/14 01:14:09] ppocr DEBUG: cls num  : 2, elapsed : 0.02582716941833496
[2024/06/14 01:14:09] ppocr DEBUG: rec_res num  : 2, elapsed : 0.14336776733398438
PEASETP.
90
[2024/06/14 01:14:10] ppocr DEBUG: dt_boxes num : 5, elapsed : 0.22837615013122559
[2024/06/14 01:14:10] ppocr DEBUG: cls num  : 5, elapsed : 0.05631256103515625
[2024/06/14 01:14:11] ppocr DEBUG: rec_res num  : 5, elapsed : 0.5270974636077881